This cleaning file results in one combined file (poverty.csv) containing data on poverty.    

Data source: https://www.census.gov/programs-surveys/saipe/data/datasets.html

In [1]:
import os
import sys
import requests
import pandas as pd


current_dir =  os.getcwd()

grandparent_dir = os.path.dirname(os.path.dirname(current_dir))
sys.path.insert(0, grandparent_dir)

In [17]:

dataframes = []

for year in range(1998, 2021 + 1):
 
    api_url = f"https://api.census.gov/data/timeseries/poverty/saipe?get=YEAR,GEOID,SAEMHI_PT,SAEPOV0_17_PT,SAEPOVRT0_17_PT,SAEPOVALL_PT,SAEPOVRTALL_PT&for=county:*&in=state:*&time={year}"
    

    response = requests.get(api_url)
    
   
    if response.status_code == 200:
       
        data = response.json()
        columns = data[0]
        data = data[1:]
        df = pd.DataFrame(data, columns=columns)
        dataframes.append(df)
        print(f"DataFrame for year {year} created successfully.")

poverty_raw = pd.concat(dataframes, ignore_index=True)

DataFrame for year 1998 created successfully.
DataFrame for year 1999 created successfully.
DataFrame for year 2000 created successfully.
DataFrame for year 2001 created successfully.
DataFrame for year 2002 created successfully.
DataFrame for year 2003 created successfully.
DataFrame for year 2004 created successfully.
DataFrame for year 2005 created successfully.
DataFrame for year 2006 created successfully.
DataFrame for year 2007 created successfully.
DataFrame for year 2008 created successfully.
DataFrame for year 2009 created successfully.
DataFrame for year 2010 created successfully.
DataFrame for year 2011 created successfully.
DataFrame for year 2012 created successfully.
DataFrame for year 2013 created successfully.
DataFrame for year 2014 created successfully.
DataFrame for year 2015 created successfully.
DataFrame for year 2016 created successfully.
DataFrame for year 2017 created successfully.
DataFrame for year 2018 created successfully.
DataFrame for year 2019 created su

In [18]:
poverty_raw.rename(columns={'YEAR': 'Year', 'GEOID': 'GeoFIPS', 'SAEMHI_PT': 'medianHouseholdIncome',
'SAEPOV0_17_PT': 'povertyUnder18', 'SAEPOVRTALL_PT':'povertyAllprct', 'SAEPOVALL_PT':'povertyAll',
'SAEPOVRT0_17_PT': 'povertyUnder18prct'
    }, inplace=True)

columns_to_keep = ['GeoFIPS', 'Year', 'povertyAll', 'povertyAllprct', 
        'povertyUnder18', 'povertyUnder18prct', 'medianHouseholdIncome']
poverty_raw = poverty_raw.loc[:, columns_to_keep]

In [19]:
from cities.utils.data_grabber import DataGrabber

data = DataGrabber()
data.get_features_wide(["gdp"])
gdp = data.wide["gdp"]

import numpy as np

In [28]:
poverty = poverty_raw.copy()

poverty['GeoFIPS'] = poverty['GeoFIPS'].astype(int)

poverty = poverty.merge(gdp[['GeoFIPS', 'GeoName']], on='GeoFIPS', how='left', suffixes=('_poverty', '_gdp'))


In [29]:
desired_order = ['GeoFIPS', 'GeoName', 'Year', 'povertyAll', 'povertyAllprct',
                  'povertyUnder18', 'povertyUnder18prct', 'medianHouseholdIncome' ]

poverty = poverty[desired_order]
poverty

,GeoFIPS,GeoName,Year,povertyAll,povertyAllprct,povertyUnder18,povertyUnder18prct,medianHouseholdIncome
0,1001,"Autauga, AL",1998,4901,11.3,2149,17.7,38260
1,1003,"Baldwin, AL",1998,15089,11.0,5688,16.8,37162
2,1005,"Barbour, AL",1998,5795,22.7,2183,31.3,27603
3,1007,"Bibb, AL",1998,3313,16.9,1279,23.7,29692
4,1009,"Blount, AL",1998,6176,12.9,2313,19.8,35623
...,...,...,...,...,...,...,...,...
75399,56037,"Sweetwater, WY",2021,3761,9.2,990,9.7,74677
75400,56039,"Teton, WY",2021,1392,5.9,223,5.5,102709
75401,56041,"Uinta, WY",2021,1917,9.4,635,11.2,70162
75402,56043,"Washakie, WY",2021,773,10.2,195,12.5,62176


In [36]:
# save them as seperate files

columns_to_save = poverty.columns[poverty.columns.get_loc("Year") + 1 :]

for column in columns_to_save:

    selected_columns = ["GeoFIPS", "GeoName", "Year", column]
    subsetpoverty = poverty[selected_columns]
    wide_df = subsetpoverty.pivot(index=['GeoFIPS', 'GeoName'], columns='Year', values= column).reset_index()

    file_name_wide = f"{column}_wide.csv"
    wide_df.to_csv(f"{grandparent_dir}/data/raw/{file_name_wide}", index=False)